# CONITEC Web scraping

*CONITEC is the National Commission for Technology Incorporation at SUS, created in 2011 by the Law nº 12.401, which addresses the therapeutic assistance and the incorporation of health technology in the scope of the Unified Health System (SUS).*

The script gets the files with the evaluation status of all requests submitted to CONITEC 

## Steps

*   CONITEC Web Scraping
*   Download Files GDRIVE

In [ ]:
pip install requests beautifulsoup4

In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup

**Aux functions**

1.   save_file_gdrive
2.   cleanStr

In [ ]:
def save_file_gdrive(file_path_gdrive, file_url):
   r = requests.get(file_url, stream = True)

   with open(file_path_gdrive, "wb") as file:  
     for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)  

In [ ]:
def cleanStr(valor):
  return (valor.replace('\n', '').replace('/','.').replace(';',',').replace('\xa0',''))[0:150]

**GET Incorporacoes**

1.   Read "Incorporacoes" from website using Beautiful soup
2.   Create a Dataframe for later manipulation
3.   Download files to Gdrive

In [ ]:
# GET html Incorporações
incorporacoesHTML = BeautifulSoup(requests.get('http://conitec.gov.br/decisoes-sobre-incorporacao-ordem-alfabetica').text, 'html.parser')

In [ ]:
#Incorporações
conteudo2 = incorporacoesHTML.findAll("table", {"class": "MsoNormalTable"})[0].findAll("tr")[3:]

# Create dictiomary + Dataframe
tecnologiaAvaliada, relatorioRecomendacao, decisao, portaria, pdf  = [], [], [], [], []

for tr in conteudo2:
  td = tr.findAll("td")
      
  if len(td) == 4 :   
    tecnologiaAvaliada.append(td[0].text.replace('\n', ''))
    relatorioRecomendacao.append(cleanStr(td[1].text))
    pdf.append('http://conitec.gov.br' + td[1].find("a")["href"])
    decisao.append(td[2].text.replace('\n', ''))
    portaria.append(td[3].text.replace('\n', ''))
 
Incorporacoes = pd.DataFrame({'TecnologiaAvaliada': tecnologiaAvaliada, 'RelatorioRecomendacao': relatorioRecomendacao, 'Decisao': decisao, 'Portaria': portaria, 'Pdf': pdf })

In [ ]:
Incorporacoes

In [ ]:
# Open GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
folder = "files_conitec/"
for item, rows in Incorporacoes.iterrows():
  file_path_gdrive = "/content/gdrive/My Drive/" +  folder + rows["RelatorioRecomendacao"] + ".pdf"
  file_url = rows["Pdf"]
  
  save_file_gdrive(file_path_gdrive, file_url)

  print("New File: ", file_path_gdrive)